In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm, skew
#from scipy.special import boxcox1px
from scipy.stats import boxcox_normmax
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scipy.special as ss

In [3]:
%matplotlib inline

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
# train = train[(train.GrLivArea < 4500) & (train.SalePrice < 700_000)]
train.reset_index(drop=True, inplace=True)
# train["SalePrice"] = np.log1p(train["SalePrice"])
y = train['SalePrice'].reset_index(drop=True).copy()

In [6]:
test_ID = test.Id

In [7]:
train.drop('Id',inplace=True, axis=1)
test.drop('Id',inplace=True, axis=1)

In [8]:
print(train.shape, test.shape)

(1460, 80) (1459, 79)


In [9]:
data = pd.concat((train,test)).reset_index(drop=True)
print(data.shape)

(2919, 80)


In [10]:
data_nans = data.isnull().sum()
data_nans = data_nans[data_nans > 0]
data_nans.sort_values(ascending=False)

PoolQC          2909
MiscFeature     2814
Alley           2721
Fence           2348
SalePrice       1459
FireplaceQu     1420
LotFrontage      486
GarageQual       159
GarageCond       159
GarageFinish     159
GarageYrBlt      159
GarageType       157
BsmtExposure      82
BsmtCond          82
BsmtQual          81
BsmtFinType2      80
BsmtFinType1      79
MasVnrType        24
MasVnrArea        23
MSZoning           4
BsmtFullBath       2
BsmtHalfBath       2
Utilities          2
Functional         2
Electrical         1
BsmtUnfSF          1
Exterior1st        1
Exterior2nd        1
TotalBsmtSF        1
GarageCars         1
BsmtFinSF2         1
BsmtFinSF1         1
KitchenQual        1
SaleType           1
GarageArea         1
dtype: int64

In [11]:
strings = ['MSSubClass']
for var in strings:
    data[var] = data[var].apply(str)

In [12]:
groups = ['Exterior1st','Exterior2nd','SaleType','Electrical','KitchenQual']
for group in groups:
    mode = data[group].mode()[0]
    data[group] = data[group].fillna(mode)

    
data.MSZoning = data.MSZoning.fillna('RL')

for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','BsmtQual',
            'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',"PoolQC"
           ,'Alley','Fence','MiscFeature','FireplaceQu','MasVnrType','Utilities']:
    data[col] = data[col].fillna('None')
    
for col in ('GarageArea', 'GarageCars','MasVnrArea','BsmtFinSF1','BsmtFinSF2'
           ,'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BsmtUnfSF','TotalBsmtSF', 'GarageYrBlt'):
    data[col] = data[col].fillna(0)
    

data.Functional = data.Functional.fillna('Typ')

data['LotFrontage'] = data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

In [13]:
data_nans = data.isnull().sum()
data_nans = data_nans[data_nans > 0]
data_nans.sort_values(ascending=False)

SalePrice    1459
dtype: int64

In [14]:
data.drop(['SalePrice'], axis=1, inplace=True)

In [15]:
cat_features = data.select_dtypes(include=['object']).columns
print(cat_features)
num_features = data.select_dtypes(exclude=['object']).columns
print(num_features)

Index(['Alley', 'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'BsmtQual', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd',
       'Fence', 'FireplaceQu', 'Foundation', 'Functional', 'GarageCond',
       'GarageFinish', 'GarageQual', 'GarageType', 'Heating', 'HeatingQC',
       'HouseStyle', 'KitchenQual', 'LandContour', 'LandSlope', 'LotConfig',
       'LotShape', 'MSSubClass', 'MSZoning', 'MasVnrType', 'MiscFeature',
       'Neighborhood', 'PavedDrive', 'PoolQC', 'RoofMatl', 'RoofStyle',
       'SaleCondition', 'SaleType', 'Street', 'Utilities'],
      dtype='object')
Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'EnclosedPorch', 'Fireplaces', 'FullBath', 'GarageArea', 'GarageCars',
       'GarageYrBlt', 'GrLivArea', 'HalfBath', 'KitchenAbvGr', 'LotArea',
       'LotFrontage', 'LowQua

### Skewing

In [16]:
feat_num = data[num_features].drop(['GarageYrBlt','MoSold','OverallCond',
                                    'OverallQual','TotRmsAbvGrd','YearBuilt',
                                    'YearRemodAdd','YrSold','BsmtFullBath','BsmtHalfBath',
                                   'FullBath','HalfBath','KitchenAbvGr','Fireplaces','GarageCars'],axis=1)
feat_cat = data[cat_features]

In [17]:
skewness = feat_num.apply(lambda x: skew(x))
skewness.sort_values(ascending=False)

MiscVal          21.947195
PoolArea         16.898328
LotArea          12.822431
LowQualFinSF     12.088761
3SsnPorch        11.376065
BsmtFinSF2        4.146143
EnclosedPorch     4.003891
ScreenPorch       3.946694
MasVnrArea        2.613592
OpenPorchSF       2.535114
WoodDeckSF        1.842433
LotFrontage       1.505704
1stFlrSF          1.469604
BsmtFinSF1        1.425230
GrLivArea         1.269358
TotalBsmtSF       1.156894
BsmtUnfSF         0.919339
2ndFlrSF          0.861675
BedroomAbvGr      0.326324
GarageArea        0.239257
dtype: float64

In [18]:
skewness = skewness[abs(skewness) > 0.5]
skewed_features = skewness.index
for feat in skewed_features:
    feat_num[feat] = ss.boxcox1p(feat_num[feat], boxcox_normmax(feat_num[feat] + 1))

In [19]:
skewness = feat_num.apply(lambda x: skew(x))
skewness = skewness[abs(skewness) > 0.5]

print("There are {} skewed numerical features after Box Cox transform".format(skewness.shape[0]))
print("Mean skewnees: {}".format(np.mean(skewness)))
skewness.sort_values(ascending=False)

There are 11 skewed numerical features after Box Cox transform
Mean skewnees: 4.431532191072031


PoolArea         14.985994
3SsnPorch         8.865144
LowQualFinSF      8.495602
MiscVal           5.239894
ScreenPorch       3.153911
BsmtFinSF2        2.579468
EnclosedPorch     2.150157
MasVnrArea        0.976354
2ndFlrSF          0.894536
WoodDeckSF        0.784768
OpenPorchSF       0.621025
dtype: float64

In [20]:
for col in feat_num.columns:
    data[col] = feat_num[col]

In [21]:
data['TotalSF'] = data.TotalBsmtSF + data['1stFlrSF'] + data['2ndFlrSF']
data['TotalBath'] = data.FullBath + 0.5 * data.HalfBath + data.BsmtFullBath + 0.5 * data.BsmtHalfBath
data['TotalPorch'] = data.OpenPorchSF + data['3SsnPorch'] + data.EnclosedPorch + data.ScreenPorch + data.WoodDeckSF

# data['YrBltAndRemod']=data.YearBuilt + data.YearRemodAdd


# data['TotYear'] = data.YearBuilt + data.YearRemodAdd + data.YrSold
data['HasPool'] = data.PoolArea.apply(lambda x: 1 if x > 0 else 0)
data['HasGarage'] = data.GarageArea.apply(lambda x: 1 if x > 0 else 0)
data['HasBsmt'] = data.TotalBsmtSF.apply(lambda x: 1 if x > 0 else 0)
data['HasFirePl'] = data.Fireplaces.apply(lambda x: 1 if x > 0 else 0)
data['BeforeCrisis'] = data.YrSold.apply(lambda x: 1 if x < 2008 else 0)

data.shape

(2919, 87)

In [22]:
qual_dict = {"None": 0, "Po": 1, "Fa": 2, "TA": 4, "Gd": 7, "Ex": 11}
qual_cols = ["ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "HeatingQC", 
             "KitchenQual", "FireplaceQu", "GarageQual", "GarageCond", "PoolQC"]

for cat in data.columns:
    if cat in qual_cols:
        data[cat] = data[cat].map(qual_dict).astype('int64')

In [23]:
drops = ['Utilities', 'Street', 'PoolQC', 'MiscFeature', 'Alley', 'Fence','GarageYrBlt']
data = data.drop(drops,axis=1)

In [24]:
cat_features = data.select_dtypes(include=['object']).columns
print(cat_features)
num_features = data.select_dtypes(exclude=['object']).columns
print(num_features)

Index(['BldgType', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'CentralAir', 'Condition1', 'Condition2', 'Electrical', 'Exterior1st',
       'Exterior2nd', 'Foundation', 'Functional', 'GarageFinish', 'GarageType',
       'Heating', 'HouseStyle', 'LandContour', 'LandSlope', 'LotConfig',
       'LotShape', 'MSSubClass', 'MSZoning', 'MasVnrType', 'Neighborhood',
       'PavedDrive', 'RoofMatl', 'RoofStyle', 'SaleCondition', 'SaleType'],
      dtype='object')
Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtCond',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtQual',
       'BsmtUnfSF', 'EnclosedPorch', 'ExterCond', 'ExterQual', 'FireplaceQu',
       'Fireplaces', 'FullBath', 'GarageArea', 'GarageCars', 'GarageCond',
       'GarageQual', 'GrLivArea', 'HalfBath', 'HeatingQC', 'KitchenAbvGr',
       'KitchenQual', 'LotArea', 'LotFrontage', 'LowQualFinSF', 'MasVnrArea',
       'MiscVal', 'MoSold', 'OpenPorchSF', 'OverallCond', 'OverallQual',

In [25]:
feat_num = data[num_features]
feat_cat = data[cat_features]

In [26]:
# features_out = pd.get_dummies(feat_cat.iloc[:len(y),:]).reset_index(drop=True)

In [27]:
# import statsmodels.api as sm

In [28]:
# Y = np.log1p(y)

In [29]:
# ols=sm.OLS(endog = Y, exog = features_out)

In [30]:
# fit = ols.fit()

In [31]:
# outlines=fit.outlier_test('holm')

In [32]:
# outlines[outlines['holm(p)']<0.01]

In [33]:
# features_out_num = feat_num.iloc[:len(y),:].reset_index(drop=True)

In [34]:
# ols_num=sm.OLS(endog = Y, exog = features_out_num)

In [35]:
# fit = ols_num.fit()

In [36]:
# outlines_num=fit.outlier_test('holm')

In [37]:
# outlines_num[outlines_num['holm(p)']<0.01]

In [38]:
# out_1 = outlines[outlines['holm(p)']<0.01].index
# out_2 = outlines_num[outlines_num['holm(p)']<0.01].index
# out = set(out_1.append(out_2))

In [39]:
out = [30, 495, 523, 632, 812, 968, 1182, 1298, 1324, 1423]

In [40]:
feat_num.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
1stFlrSF,2919.0,5.467018,0.192349,4.727521,5.332384,5.459344,5.606361,6.333169
2ndFlrSF,2919.0,401.452043,514.288936,0.000000,0.000000,0.000000,835.536971,2532.646194
3SsnPorch,2919.0,0.080091,0.711485,0.000000,0.000000,0.000000,0.000000,7.814872
BedroomAbvGr,2919.0,2.860226,0.822693,0.000000,2.000000,3.000000,3.000000,8.000000
BsmtCond,2919.0,3.936622,0.989886,0.000000,4.000000,4.000000,4.000000,7.000000
BsmtFinSF1,2919.0,72.169058,63.053998,0.000000,0.000000,75.846052,120.877939,475.812603
BsmtFinSF2,2919.0,1.159759,3.245977,0.000000,0.000000,0.000000,0.000000,14.376852
BsmtFullBath,2919.0,0.429599,0.524676,0.000000,0.000000,0.000000,1.000000,3.000000
BsmtHalfBath,2919.0,0.061322,0.245608,0.000000,0.000000,0.000000,0.000000,2.000000
BsmtQual,2919.0,5.689962,2.401908,0.000000,4.000000,7.000000,7.000000,11.000000


In [41]:
feat_num = feat_num.apply(lambda x: (x - np.mean(x)) / np.std(x)).reset_index(drop=True)

In [42]:
feat_num.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtCond,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtQual,...,YearRemodAdd,YrSold,TotalSF,TotalBath,TotalPorch,HasPool,HasGarage,HasBsmt,HasFirePl,BeforeCrisis
0,-0.772873,1.202038,-0.112588,0.169927,0.064036,0.724690,-0.357352,1.087334,-0.249719,0.545509,...,0.896833,0.157646,1.126208,1.584766,-0.762201,-0.066884,0.239219,0.166784,-1.027440,-0.902939
1,0.434763,-0.780730,-0.112588,0.169927,0.064036,1.183898,-0.357352,-0.818929,3.822508,0.545509,...,-0.395604,-0.602962,-0.672343,0.348216,1.001211,-0.066884,0.239219,0.166784,0.973292,1.107495
2,-0.545974,1.230753,-0.112588,0.169927,0.064036,0.307824,-0.357352,1.087334,-0.249719,0.545509,...,0.848965,0.157646,1.191082,1.584766,-0.848140,-0.066884,0.239219,0.166784,0.973292,-0.902939
3,-0.409344,0.968004,-0.112588,0.169927,3.095207,-0.307610,-0.357352,1.087334,-0.249719,-0.703712,...,-0.682812,-1.363569,0.831604,-0.270058,-0.248083,-0.066884,0.239219,0.166784,0.973292,1.107495
4,0.135282,1.679725,-0.112588,1.385655,0.064036,0.632444,-0.357352,1.087334,-0.249719,0.545509,...,0.753229,0.157646,1.769821,1.584766,1.008861,-0.066884,0.239219,0.166784,0.973292,-0.902939


In [43]:
cat_data = pd.get_dummies(feat_cat).reset_index(drop=True)

In [44]:
fin_data = pd.concat([feat_num,cat_data],axis=1).reset_index(drop=True)

In [45]:
fin_data.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtCond,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtQual,...,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,-0.772873,1.202038,-0.112588,0.169927,0.064036,0.724690,-0.357352,1.087334,-0.249719,0.545509,...,0,0,0,0,0,0,0,0,0,1
1,0.434763,-0.780730,-0.112588,0.169927,0.064036,1.183898,-0.357352,-0.818929,3.822508,0.545509,...,0,0,0,0,0,0,0,0,0,1
2,-0.545974,1.230753,-0.112588,0.169927,0.064036,0.307824,-0.357352,1.087334,-0.249719,0.545509,...,0,0,0,0,0,0,0,0,0,1
3,-0.409344,0.968004,-0.112588,0.169927,3.095207,-0.307610,-0.357352,1.087334,-0.249719,-0.703712,...,0,0,0,0,0,0,0,0,0,1
4,0.135282,1.679725,-0.112588,1.385655,0.064036,0.632444,-0.357352,1.087334,-0.249719,0.545509,...,0,0,0,0,0,0,0,0,0,1


In [46]:
print(fin_data.shape)
train = fin_data.iloc[:len(y),:]
test = fin_data.iloc[len(y):,:]
print(train.shape,y.shape,test.shape)

(2919, 266)
(1460, 266) (1460,) (1459, 266)


In [47]:
outliers = list(out)
train = train.drop(train.index[outliers])
y_train = y.drop(y.index[outliers])

In [48]:
print(train.shape,y_train.shape,test.shape)

(1450, 266) (1450,) (1459, 266)


In [49]:
train['SalePrice'] = y_train

In [50]:
print(train.shape)

(1450, 267)


In [51]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.version)

<module 'tensorflow_core._api.v2.version' from '/home/romas/anaconda3/lib/python3.7/site-packages/tensorflow_core/_api/v2/version/__init__.py'>


In [52]:
train_set = train.sample(frac=0.8, random_state=0)
test_set = train.drop(train_set.index)

In [53]:
train_labels = train_set.pop('SalePrice')
test_labels = test_set.pop('SalePrice')

In [54]:
train_stats = train_set.describe()
train_stats = train_stats.transpose()
train_stats.head()

,count,mean,std,min,25%,50%,75%,max
1stFlrSF,1160.0,0.034547,0.961412,-3.845208,-0.643090,-0.001456,0.734413,3.218656
2ndFlrSF,1160.0,0.028160,1.020174,-0.780730,-0.780730,-0.780730,0.927498,3.670955
3SsnPorch,1160.0,0.035648,1.154146,-0.112588,-0.112588,-0.112588,-0.112588,10.873180
BedroomAbvGr,1160.0,0.001192,0.981000,-3.477258,-1.045801,0.169927,0.169927,6.248569
BsmtCond,1160.0,0.036164,0.989298,-3.977524,0.064036,0.064036,0.064036,3.095207


In [55]:
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[len(train_set.keys())]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae', 'mse'])
    return model

In [56]:
model = build_model()

In [57]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                17088     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 21,313
Trainable params: 21,313
Non-trainable params: 0
_________________________________________________________________


In [58]:
normed_train_data = train_set
normed_test_data = test_set

In [59]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

array([[ 0.3453611 ],
       [-0.6804416 ],
       [-1.0268404 ],
       [-0.44266897],
       [-0.31040564],
       [ 0.28511095],
       [-0.35449755],
       [-0.70969677],
       [-0.9466952 ],
       [-0.84824616]], dtype=float32)

In [60]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: 
            print('')
            print(epoch)
        print('.', end='')


In [61]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [SalePrice]')
    plt.plot(hist['epoch'], hist['mae'],label='Train Error')
    plt.plot(hist['epoch'], hist['val_mae'],label = 'Val Error')
    plt.ylim([10000,30_000])
    plt.legend()

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$SalePrice$]')
    plt.plot(hist['epoch'], hist['mse'],label='Train Error')
    plt.plot(hist['epoch'], hist['val_mse'],label = 'Val Error')
    plt.ylim([5e6,1e9])
    plt.legend()
    plt.show()

In [ ]:
model = build_model()

EPOCHS = 5000

# Параметр patience определяет количество эпох, проверяемых на улучшение
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])




0
....................................................................................................
100
....................................................................................................
200
..........................

In [ ]:
plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f}".format(mae))


In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [log(Price)]')
plt.ylabel('Predictions [log(Price)]')
plt.axis('equal')
plt.axis('square')
plt.xlim([10,plt.xlim()[1]])
plt.ylim([10,plt.ylim()[1]])
_ = plt.plot([-100_000_000, 100_000_000], [-100_000_000, 100_000_000])


In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [Price]")
_ = plt.ylabel("Count")


In [ ]:
result = model.predict(test)

In [ ]:
final_sub = pd.DataFrame()
final_sub['Id'] = test_ID
final_sub['SalePrice'] = result

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = result
sub.to_csv('2020-02-13 NN sub tf2x + skew.csv',index=False)

In [ ]:
bs = pd.read_csv('best_submission.csv')

In [ ]:
final_sub['BSP'] = bs.SalePrice
final_sub

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
np.sqrt(mean_squared_error(final_sub.SalePrice, final_sub.BSP))

In [ ]:
mean_absolute_error(final_sub.SalePrice, final_sub.BSP)